In [36]:
!pip install qiskit qiskit_aer matplotlib

In [37]:
!pip install pylatexenc


In [38]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector
import numpy as np
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram


In [39]:
#Vary the number of qubits
def qft_rotations(qc, n):

    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(-np.pi / 2 ** (n - qubit), qubit, n)

    qft_rotations(qc, n)
    return qc

In [40]:
def swap_registers(qc, n):

    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc


def qft_circuit(n):

    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc


In [41]:
#Inverse QFT
def inverse_qft_rotations(qc, n):

    if n == 0:
        return qc
    n -= 1
    for qubit in range(n):
        qc.cp(-np.pi / 2 ** (n - qubit), qubit, n)
    qc.h(n)
    inverse_qft_rotations(qc, n)
    return qc


def inverse_qft_circuit(n):

    qc = QuantumCircuit(n)
    swap_registers(qc, n)
    inverse_qft_rotations(qc, n)
    qc.name = "Inverse QFT"
    return qc

In [42]:
def run_qft(n):

    qc = qft_circuit(n)

    simulator = AerSimulator(method='statevector')
    qc.save_statevector()
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled).result()
    statevector = result.data(0)["statevector"]

    print("\nQFT Circuit:")
    print(qc.draw(output='text'))
    plot_bloch_multivector(statevector)
    plt.show()

In [43]:
#Measure Output States

def measure_qft(n):

    qc = qft_circuit(n)
    qc.measure_all()

    simulator = AerSimulator()
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=1024).result()
    counts = result.get_counts()

    print("\nMeasured QFT Circuit:")
    print(qc.draw(output='text'))
    plot_histogram(counts)
    plt.show()

In [44]:
#Integration with Phase Estimation

def simple_phase_estimation():

    qc = QuantumCircuit(4, 3)

    # Apply Hadamard to counting qubits
    qc.h(range(3))

    # Apply controlled-U (T gate = phase π/4)
    for i in range(3):
        qc.cp(np.pi / 4 * (2 ** i), i, 3)

    # Apply inverse QFT on counting qubits
    iqft = inverse_qft_circuit(3)
    qc.append(iqft.to_instruction(), range(3))

    qc.measure(range(3), range(3))

    simulator = AerSimulator()
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=1024).result()
    counts = result.get_counts()

    print("\nPhase Estimation Circuit:")
    print(qc.draw(output='text'))
    plot_histogram(counts)
    plt.show()

In [45]:
!pip install qiskit qiskit-aer pylatexenc matplotlib


In [47]:
#  Main Execution Block for Google Colab
if __name__ == "__main__":
    # Setup (ensures visuals display correctly in Colab)
    %matplotlib inline

    #  Vary the number of qubits
    for n in [2, 3, 4]:
        print(f"\n Running QFT with {n} qubits ")
        run_qft(n)

    #  Inverse QFT demo
    print("\n Demonstrating Inverse QFT on 3 qubits:")
    iqft = inverse_qft_circuit(3)
    print(iqft.draw(output='text'))

    #  Run QFT with measurements
    print("\n Measuring output of 3-qubit QFT:")
    measure_qft(3)

    #  Integrate with Phase Estimation
    print("\n Running Phase Estimation Demo:")
    simple_phase_estimation()





 Running QFT with 2 qubits 

QFT Circuit:
                    ┌───┐    statevector 
q_0: ──────■────────┤ H ├─X───────░──────
     ┌───┐ │P(-π/2) └───┘ │       ░      
q_1: ┤ H ├─■──────────────X───────░──────
     └───┘                        ░      

 Running QFT with 3 qubits 

QFT Circuit:
                                             ┌───┐    statevector 
q_0: ──────■────────────────────────■────────┤ H ├─X───────░──────
           │                  ┌───┐ │P(-π/2) └───┘ │       ░      
q_1: ──────┼─────────■────────┤ H ├─■──────────────┼───────░──────
     ┌───┐ │P(-π/4)  │P(-π/2) └───┘                │       ░      
q_2: ┤ H ├─■─────────■─────────────────────────────X───────░──────
     └───┘                                                 ░      

 Running QFT with 4 qubits 

QFT Circuit:
                                                                      »
q_0: ──────■──────────────────────────────────■───────────────────────»
           │                                  │ 